###### Documentação parquet
https://spark.apache.org/docs/latest/sql-data-sources-parquet.html 

https://spark.apache.org/docs/latest/sql-data-sources-parquet.html

In [0]:
dfpq = spark.read.parquet("/FileStore/tables/Anac/V_OCORRENCIA_AMPLAParquet")
display(dfpq)

In [0]:
#verificando dados distintos de colunas com spark sql 
display(dfpq.select("Classificacao_da_Ocorrência").distinct())


In [0]:
dfpq.write \
    .partitionBy("Classificacao_da_Ocorrência") \
    .mode("overwrite") \
    .parquet("/FileStore/tables/Anac/parquet_particionado")

In [0]:
display(dbutils.fs.ls("dbfs:/FileStore/tables/Anac"))

######Vendo Arquivos particionados

In [0]:
display(dbutils.fs.ls("dbfs:/FileStore/tables/Anac/parquet_particionado/"))

In [0]:
#Descendo mais um nivel ver os logs de cada partição
display(dbutils.fs.ls("dbfs:/FileStore/tables/Anac/parquet_particionado/Classificacao_da_Ocorrência=Acidente/"))


######lendo arquivo particionado 

In [0]:
dfacidente = spark.read.parquet("/FileStore/tables/Anac/parquet_particionado/Classificacao_da_Ocorrência=Ocorrência de Solo/")
display(dfacidente)

######Ler todos os dados mesmo particionados (agrupados)

In [0]:
dfTudo = spark.read\
    .format("parquet")\
    .load("dbfs:/FileStore/tables/Anac/parquet_particionado/")
display(dfTudo)

######Salvando em mais de 1 partição 

In [0]:
#exemplo , uma equipe de cada estado analisa as "Classificacao da Ocorrência" e cada uma das ocorrencias é analizado por uma pessoa
#Separarar (particionar por Classificacao da Ocorrência e estado)


dfpq.write \
    .partitionBy("UF","Classificacao_da_Ocorrência") \
    .mode("overwrite") \
    .parquet("/FileStore/tables/Anac/parquet_Multiparticionado")

#Obs: pode demorar na escrita devido aos particionamentos , mas ganha tempo na leitura 


In [0]:
display(dbutils.fs.ls("dbfs:/FileStore/tables/Anac"))

In [0]:
display(dbutils.fs.ls("/FileStore/tables/Anac/parquet_Multiparticionado/"))

In [0]:
display(dbutils.fs.ls("/FileStore/tables/Anac/parquet_Multiparticionado/UF=MG/"))

In [0]:
display(dbutils.fs.ls("/FileStore/tables/Anac/parquet_Multiparticionado/UF=MG/Classificacao_da_Ocorrência=Acidente/"))

In [0]:
#simulando o responsavel de Mg onde a Classificacao da Ocorrência seja igual a Acidente trabalharia com o arquivo 
#dbfs:/FileStore/tables/Anac/parquet_Multiparticionado/UF=MG/Classificacao_da_Ocorrência=Acidente/
#e nao todo o arquivo original , ganhando performace na leitura 

Dados Acidente do analista de MG

In [0]:

DfAcidentesMG = spark.read\
    .format("parquet")\
    .load("/FileStore/tables/Anac/parquet_Multiparticionado/UF=MG/Classificacao_da_Ocorrência=Acidente/")
display(DfAcidentesMG)
